In [13]:
HSI_data.shape

(307, 307, 191)

In [16]:
!pip install HSI2RGB

ERROR: Could not find a version that satisfies the requirement HSI2RGB
ERROR: No matching distribution found for HSI2RGB


In [19]:
import numpy as np
import scipy.io as spio
from scipy.interpolate import PchipInterpolator
from bisect import bisect

def HSI2RGB(wY,HSI,ydim,xdim,d,threshold):
# wY: wavelengths in nm
# Y : HSI as a (#pixels x #bands) matrix,
# dims: x & y dimension of image
# d: 50, 55, 65, 75, determines the illuminant used, if in doubt use d65
# thresholdRGB : True if thesholding should be done to increase contrast
#
#
# If you use this method, please cite the following paper:
#  M. Magnusson, J. Sigurdsson, S. E. Armansson, M. O. Ulfarsson, 
#  H. Deborah and J. R. Sveinsson, 
#  "Creating RGB Images from Hyperspectral Images Using a Color Matching Function", 
#  IEEE International Geoscience and Remote Sensing Symposium, Virtual Symposium, 2020
#
#  @INPROCEEDINGS{hsi2rgb, 
#  author={M. {Magnusson} and J. {Sigurdsson} and S. E. {Armansson} 
#  and M. O. {Ulfarsson} and H. {Deborah} and J. R. {Sveinsson}}, 
#  booktitle={IEEE International Geoscience and Remote Sensing Symposium}, 
#  title={Creating {RGB} Images from Hyperspectral Images using a Color Matching Function}, 
#  year={2020}, volume={}, number={}, pages={}}
#
# Paper is available at
# https://www.researchgate.net/profile/Jakob_Sigurdsson
#
#

    
    # Load reference illuminant
    D = spio.loadmat('./D_illuminants.mat')
    w = D['wxyz'][:,0]
    x = D['wxyz'][:,1]
    y = D['wxyz'][:,2]
    z = D['wxyz'][:,3]
    D = D['D']
    
    i = {50:2,
         55:3,
         65:1,
         75:4}
    wI = D[:,0];
    I = D[:,i[d]];
        
    # Interpolate to image wavelengths
    I = PchipInterpolator(wI,I,extrapolate=True)(wY) # interp1(wI,I,wY,'pchip','extrap')';
    x = PchipInterpolator(w,x,extrapolate=True)(wY) # interp1(w,x,wY,'pchip','extrap')';
    y = PchipInterpolator(w,y,extrapolate=True)(wY) # interp1(w,y,wY,'pchip','extrap')';
    z = PchipInterpolator(w,z,extrapolate=True)(wY) # interp1(w,z,wY,'pchip','extrap')';

    # Truncate at 780nm
    i=bisect(wY, 780)
    HSI=HSI[:,0:i]/HSI.max()
    wY=wY[:i]
    I=I[:i]
    x=x[:i]
    y=y[:i]
    z=z[:i]
    
    # Compute k
    k = 1/np.trapz(y * I, wY)
    
    # Compute X,Y & Z for image
    X = k * np.trapz(HSI @ np.diag(I * x), wY, axis=1)
    Z = k * np.trapz(HSI @ np.diag(I * z), wY, axis=1)
    Y = k * np.trapz(HSI @ np.diag(I * y), wY, axis=1)
    
    XYZ = np.array([X, Y, Z])
    
    # Convert to RGB
    M = np.array([[3.2404542, -1.5371385, -0.4985314],
                  [-0.9692660, 1.8760108, 0.0415560],
                  [0.0556434, -0.2040259, 1.0572252]]);
    sRGB=M@XYZ;
    
    # Gamma correction
    gamma_map = sRGB >  0.0031308;
    sRGB[gamma_map] = 1.055 * np.power(sRGB[gamma_map], (1. / 2.4)) - 0.055;
    sRGB[np.invert(gamma_map)] = 12.92 * sRGB[np.invert(gamma_map)];
    # Note: RL, GL or BL values less than 0 or greater than 1 are clipped to 0 and 1.
    sRGB[sRGB > 1] = 1;
    sRGB[sRGB < 0] = 0;
    
    if threshold:
        for idx in range(3):
            y = sRGB[idx,:];
            a,b = np.histogram(y,100)
            b = b[:-1] + np.diff(b)/2
            a=np.cumsum(a)/np.sum(a)
            th = b[0]
            i = a<threshold;
            if i.any():
                th=b[i][-1];
            y=y-th
            y[y<0] = 0

            a,b=np.histogram(y,100)
            b = b[:-1] + np.diff(b)/2
            a=np.cumsum(a)/np.sum(a);
            i = a > 1-threshold
            th=b[i][0]
            y[y>th]=th
            y=y/th
            sRGB[idx,:]=y
        
    R = np.reshape(sRGB[0,:],[ydim,xdim]);
    G = np.reshape(sRGB[1,:],[ydim,xdim]);
    B = np.reshape(sRGB[2,:],[ydim,xdim]);
    return np.transpose(np.array([R,G,B]),[1,2,0])

In [61]:
D = spio.loadmat('./D_illuminants.mat')

In [62]:
D.keys()

dict_keys(['__header__', '__version__', '__globals__', 'wxyz', 'D'])

In [24]:
D

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Thu Nov  7 14:09:36 2019',
 '__version__': '1.0',
 '__globals__': [],
 'wxyz': array([[3.80000e+02, 1.36800e-03, 3.90000e-05, 6.45000e-03],
        [3.85000e+02, 2.23600e-03, 6.40000e-05, 1.05500e-02],
        [3.90000e+02, 4.24300e-03, 1.20000e-04, 2.00500e-02],
        [3.95000e+02, 7.65000e-03, 2.17000e-04, 3.62100e-02],
        [4.00000e+02, 1.43100e-02, 3.96000e-04, 6.78500e-02],
        [4.05000e+02, 2.31900e-02, 6.40000e-04, 1.10200e-01],
        [4.10000e+02, 4.35100e-02, 1.21000e-03, 2.07400e-01],
        [4.15000e+02, 7.76300e-02, 2.18000e-03, 3.71300e-01],
        [4.20000e+02, 1.34380e-01, 4.00000e-03, 6.45600e-01],
        [4.25000e+02, 2.14770e-01, 7.30000e-03, 1.03905e+00],
        [4.30000e+02, 2.83900e-01, 1.16000e-02, 1.38560e+00],
        [4.35000e+02, 3.28500e-01, 1.68400e-02, 1.62296e+00],
        [4.40000e+02, 3.48280e-01, 2.30000e-02, 1.74706e+00],
        [4.45000e+02, 3.48060e-01, 2.98000e-02,

In [63]:
D['wxyz'].shape

(81, 4)

In [64]:
D['D'].shape

(97, 5)

In [65]:
w = D['wxyz'][:,0]   # 파장이라고 불리던 것..!! 
x = D['wxyz'][:,1]
y = D['wxyz'][:,2]
z = D['wxyz'][:,3]


In [66]:
D = D['D'] # d에 해당하는 것만 추출

In [68]:
D.shape

(97, 5)

In [50]:
D[0]

array([3.00e+02, 3.41e-02, 1.90e-02, 2.40e-02, 4.30e-02])

In [69]:
y.shape

(81,)

In [73]:
wl = D[:,0]

In [75]:
i = {50:2,
         55:3,
         65:1,
         75:4}
# I = D[:,i[d]]

In [91]:
l = D[:, 1]
l

array([3.41000e-02, 1.66430e+00, 3.29450e+00, 1.17652e+01, 2.02360e+01,
       2.86447e+01, 3.70535e+01, 3.85011e+01, 3.99488e+01, 4.24302e+01,
       4.49117e+01, 4.57750e+01, 4.66383e+01, 4.93637e+01, 5.20891e+01,
       5.10323e+01, 4.99755e+01, 5.23118e+01, 5.46482e+01, 6.87015e+01,
       8.27549e+01, 8.71204e+01, 9.14860e+01, 9.24589e+01, 9.34318e+01,
       9.00570e+01, 8.66823e+01, 9.57736e+01, 1.04865e+02, 1.10936e+02,
       1.17008e+02, 1.17410e+02, 1.17812e+02, 1.16336e+02, 1.14861e+02,
       1.15392e+02, 1.15923e+02, 1.12367e+02, 1.08811e+02, 1.09082e+02,
       1.09354e+02, 1.08578e+02, 1.07802e+02, 1.06296e+02, 1.04790e+02,
       1.06239e+02, 1.07689e+02, 1.06047e+02, 1.04405e+02, 1.04225e+02,
       1.04046e+02, 1.02023e+02, 1.00000e+02, 9.81671e+01, 9.63342e+01,
       9.60611e+01, 9.57880e+01, 9.22368e+01, 8.86856e+01, 8.93459e+01,
       9.00062e+01, 8.98026e+01, 8.95991e+01, 8.86489e+01, 8.76987e+01,
       8.54936e+01, 8.32886e+01, 8.34939e+01, 8.36992e+01, 8.186

wl은 illuminants에서 65를 사용하기에 D에 해당하는 곳에서 index = 1번째 인것들에 해당하는 열만 가져와서 l에 집어넣기

In [90]:
I = PchipInterpolator(wI,l,extrapolate=True)(wY) # interp1(wI,I,wY,'pchip','extrap')';
x = PchipInterpolator(w,x,extrapolate=True)(wY) # interp1(w,x,wY,'pchip','extrap')';
y = PchipInterpolator(w,y,extrapolate=True)(wY) # interp1(w,y,wY,'pchip','extrap')';
z = PchipInterpolator(w,z,extrapolate=True)(wY) # interp1(w,z,wY,'pchip','extrap')';

NameError: name 'wY' is not defined